# Overfitting vs Underfitting

## Overfitting

When the model passes through all points, leading to 0 error on training set,
but high error on unseen data.

## Underfitting

Model doesn't pass through most of the points, thus leading to high error
metrics.

"A good model is the one whose error decreases with increase in complexity"


# Regularization

Reduces model overfitting and variance.

## Three Types

* L1 Regularization - LASSO Regression
* L2 Regularization - Ridge Regression
* L1 & L2 Regularization - Elastic Net


# Feature Scaling

It is used when we have features that are in different units.
Fitting is only to be done on training data

## Process

* Perform train-test split
* Fit to training feature data
* Transform training feature data
* Transform test feature data

DO NOT SCALE LABEL DATA.


# Cross Validation

An advanced way to split data into training and testing sets.

Train-validation-test split with K-fold is the best way to cross validate.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, SCORERS
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [2]:
df = pd.read_csv('~/Documents/Data Science Masterclass/DATA/Advertising.csv')

In [3]:
X = df.drop('sales', axis=1)

In [4]:
y = df['sales']

In [5]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [6]:
polynomial_features = polynomial_converter.fit_transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(polynomial_features, 
    y, test_size=0.3, random_state=101)

In [8]:
scaler = StandardScaler()

In [9]:
scaler.fit(X_train)

StandardScaler()

In [10]:
X_train = scaler.transform(X_train)

In [11]:
X_test = scaler.transform(X_test)

# Ridge Regression

A regularization technique that reduces potential over-fitting to the training
data by adding a penalty term to the error that is based on the squared value
of the coefficients.

* Error = RSS + µß^2
Note: µ is referred to as å in sklearn function calls.


In [12]:
ridge_model = Ridge(alpha=10)

In [13]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=10)

In [14]:
y_hat = ridge_model.predict(X_test)

In [15]:
RMSE = np.sqrt(mean_squared_error(y_test, y_hat))
RMSE

0.8946386461319685

In [16]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1, 10), scoring='neg_mean_absolute_error')

In [17]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [18]:
ridge_cv_model.alpha_ # Get best alpha value

0.1

In [19]:
y_hat = ridge_cv_model.predict(X_test)

In [20]:
RMSE = np.sqrt(mean_squared_error(y_test, y_hat))
RMSE

0.6180719926924644

# Lasso Regression

Can yield same error (or better) than ridge regression, but does so by making
the equation much simpler (as some of the coefficients will be 0)

In [21]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5)

In [22]:
lasso_cv_model.fit(X_train, y_train)

LassoCV(cv=5, eps=0.1)

In [23]:
lasso_cv_model.alpha_

0.4943070909225828

In [24]:
y_hat = ridge_model.predict(X_test)

In [25]:
RMSE = np.sqrt(mean_squared_error(y_test, y_hat))
RMSE

0.8946386461319685

In [26]:
lasso_cv_model.coef_ # Notice how only 2 parameters have non-zero coefficients,
# making the model much simpler

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

# Elastic Net
* Combination of L1 and L2
* Total Error = Difference + µ|ß| + µß^2

DIRECTLY USE ELASTIC NET INSTEAD OF L1 and L2 AS IT IS COMBINATION OF BOTH.


In [27]:
elastic_model = ElasticNetCV(l1_ratio=[0.1,  0.5, 0.7, 0.9, 0.95, 0.99, 1], 
    eps=0.001, n_alphas=100, max_iter=1000000)

In [28]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [30]:
elastic_model.l1_ratio_ # 1 -> Lasso Regression, 0 -> Ridge Regression

1.0

In [32]:
test_predictions = elastic_model.predict(X_test)

In [35]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))
RMSE

0.6063140748984036